In [1]:
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import math

In [2]:
%%latex
\newcommand{\matr}[1]\textbf{#1}
\newcommand{\vect}[1]{\vec{#1}}
\usepackage[table,xcdraw]{xcolor}

<IPython.core.display.Latex object>

# To my dear supervisors

Dear Professor Likos!
Dear Dr. Bianchi!

Professor Likos, I hope you're enjoying your vacation.

You haven't heard from me in a while because I was on vaction too, but I have started my work on the thesis. 
I will review what we discussed in person and online with Dr. Bianchi, and I hope you are as interested in the
research question as I am. Dr. Bianchi, if you have suggestions for adapting dimensions/parameters of the situation (fluid flow through 2d cross section of pipe with cylindrical obstacles) or even tweak the situation, I would love to hear your suggestions too.
I hope I depicted the situation somewhat well below. I thought it would be interesting to not only try obstacles that are
regularly spaced, all the same size, but try a more heterogeneous situation. But that is still a little ways down the road,
so I'll have to see about that.

Progress-wise I'd say I'm still at the beginning. All the work here has been done in about 1 week. I'm working on the collision step now. It took a lot of time to get familiar with C++ again, and to find out how best to test my results. I think I have found a good way to do this, called unit testing, which basically means you test every unit of your program separately and make sure they work under various circumstances, to make debugging easier later down the road and also to add more credibility to my results.

Please do not worry if the writing style is sometimes informal, and at other times formal. I meant it to be that way, as some sections are unfinished/jumbled down thoughts/still expanding/experimenting. Also, the citing style is not the final version either, unfortunately, I cannot change it at the moment.

I'd be happy to hear your guidance and if everything is looking good, or if you suggest I focus more on other areas. If you have a lot to tell me I'd be happy to meet over Zoom/Teams or in person.

Where possible, I've not cited Wikipedia, or added other sources. Is it still okay if I cite Wikipedia, or is that completely undesirable?

All the best,

Chris

# MPCD simulation of polymers in solution

This notebook will serve as the documentation of our efforts and results for my Bachelor's Thesis. The goal of this thesis will be the study of short/long-chained polymers in a liquid thats flowing around obstacles. The liquid will be simulated with MPCD, or "Multi Particle Collision Dynamics".

I chose the language C++ for its familiar object-oriented nature and its proven execution-time. Output of the simulation will mostly be analysed in Python, specifically with Jupyter Notebook for a blend of beautiful visualizations and convenience. For simplification, the situation will be studied in 2D. The situation we specifically discussed is shown below.

![Situation](Assets/MPCD_Situation.png)

### Open questions & Description of situation

For these questions, I need to expand the program to even consider them. If I get close and still have no idea how to go about them, I will contact Maximilian Liebetreu (I think he was there when we first met, I'm sorry I can't remember exactly). Are there other people I can contact for the programming/implementation details?

The fluid flows from the -x direction through the 2D cross-section of some vessel. In this vessel there are cylindrically shaped objects, through which the fluid cannot pass. (->How will I simulate the obstacles, are they solid non-deformable, are they soft & elastic, ..?) (-> Methods: create a lot of particles and immobilize them?, check for collision between particles and obstacles on a per particle basis? (this will probably be too expensive computationally))

What happens when fluid goes out of bounds? (->Should particles that go too far in the +x direction be destroyed and new ones created from the -x direction, simulating continuous flow?)

Finally, polymers will be introduced into the fluid and their behavior studied. We might imagine this "vessel" as a blood vessel and the obstacles as cells moving around in the bloodstream, for example. (-> For the polymers, you told me I should wait until the MPCD is really working)

# Introduction

Multiparticle collision dynamics (MPCD), also known as Stochastic Rotation Dynamics (SRD)[@winkl2009] is a technique originally introduced to study the dynamics of complex fluids such as polymers in solution. Besides MPCD, there exist other mesoscopic models that have been constructed for this purpose, such as Langevin, Direct Simulation Monte Carlo and lattice Boltzmann methods.[@malev1999] We only concern ourselves with the application of MPCD, it follows that any comparison between methods are out of the scope of this thesis.

The MPCD technique models the fluid using particles, their positions and velocities are treated as continuous variables. The system is divided up into cells that have no restriction on the number of particles, each of the cells is part of a regular lattice. The dynamics is split into two parts: Particle streaming and multiparticle collision dynamics. Particle streaming is treated exactly for each particle in the system, while the collision step is approximated on a cell level. The multiparticle collision dynamics conserves mass, momentum and energy and leads to the correct hydrodynamical equations.[@malev1999] The streaming and collision step are described in more detail in (TODO).

# How does MPCD work?

The system we are modelling consists of $N$ particles with mass $m$, position $\vec{r_{i}}$ and velocity $\vec{v_{i}}$, where $i \in \{1, 2, \dots, N\}$. One timestep shall correspond to having calculated all the new particle positions and velocities in the streaming and collision steps, respectively. For each of the N particles, the streaming and collision steps are applied, and this pattern is repeated until the wanted number of timesteps have elapsed.

## The streaming step

The streaming step is very straightforward. The particle positions are simply updated according to

\begin{equation}
\vect{r_{i}} \rightarrow \vect{r_{i}} + \Delta t \cdot \vect{v_{i}}\textrm{,}
\end{equation}

where $\Delta t$ is a small time interval.[@winkl2009][@malev1999]

## The collision step

The collision step is somewhat more complicated. It involves the mean velocity of all particles in a particular cell, $\vect{V_c}$, the velocity of the particle $i$ $\vec{v_i}$ and a rotation matrix $\matr{R}(\alpha)$. The vector $\vect{v_i}$ is rotated relative to the mean velocity $\vect{V_c}$ of all particles in cell $c$, cell $c$ being the cell which particle $i$ belongs to. It is shown in [@malev1999] that the rule,

\begin{equation}
\vect{v_i} \rightarrow \vect{V_c} + \matr{R}(\alpha) [\vect{v_i} - \vect{V_c}] \textrm{,}
\end{equation}

conserves mass, momentum and energy. The rotation matrix $\matr{R}(\alpha)$ is a simple 2d rotation matrix

\begin{equation}
R(\alpha) = 
\left[ \begin{array}{rr}
cos(\alpha) & -sin(\alpha) \\
sin(\alpha) & cos(\alpha) \\
\end{array}\right]
\end{equation}

It is the same for all particles of a cell, but since $\alpha$ is sampled randomly,  will probably differ from cell to cell. The mean velocity of a cell is defined as

\begin{equation}
\vect{V_c} = \frac{1}{N_c} \sum_{i=1}^{N_c} \vect{v_i} \textrm{,}
\end{equation}

where $N_c$ is the number of particles in the cell.[@malev1999]

# (Pseudo) Random Number Generation

One of the pillars of this thesis is the generation of random rotation angles for the rotation matrix needed in the collision step. This proved to be somewhat difficult. First, the standard algorithm of the C++ standard library was tried, but it didn't qualify because it performed poorly in comparison to the second and third algorithms tried, which are called "Mersenne Twister" and "xoshiro256++", respectively.[@wiki:mersennetwister][@cppreference:prng][@unimi:xoshiro]

The Mersenne Twister was implemented using the C++ standard library. The xoshiro256++ was implemented using Sebastian Vigna's code with some additions.[@unimi:xoshiro]

To compare algorithms, and also to make sure that the implementation of the xoshiro256++ is right, a $\chi^2$ test for discrete observations was used. The generated angles in the interval $[0, 2\pi)$ were split into $k+1$ buckets, where $k$ is the number of degrees of freedom of the $\chi^2$ distribution. The test error

\begin{equation}
T = \sum_{b=1}^{k+1}{\frac{(N_o - E[N_b])^2}{E[N_b]}},
\end{equation}

where $E[N_b] = \frac{N}{b}, b \in \{1, 2, \dots , k+1\}$ is the expected bucket size, is compared to $\chi^2_{1-\alpha, k}$, where $\alpha$ is the signifigance level. The null hypothesis

$$
H_0: \textrm{The angles are distributed uniformly in the interval } [0, 2 \pi)
$$

is tested against the alternative hypothesis

$$
H_1: \textrm{The angles are not distributed uniformly in the interval } [0, 2 \pi) \textrm{.}
$$

If the test should have significance level $\alpha$, $H_0$ is rejected if $T \ge \chi^2_{1-\alpha, k}$.[@fruehwirthstat][@wiki:chisquaredtest][@wiki:goodnessoffit]

The results of the $\chi^2$ test are summarised in [TODO: Table, and table formatting].

In [3]:
path = "../x64/Debug/Data/RNG/"
mersenne = "mersenne_twister_chi2.csv"
xoshiro = "xoshiro_chi2.csv"
index = ["k", "chi^2 probability"]
out_path = "Generated/"
new_name = "chi2_results_dirty.csv"
mersenne_twister_chi2 = pd.read_csv(path + mersenne).set_index(index)
xoshiro256plusplus_chi2 = pd.read_csv(path + xoshiro).set_index(index)
results = mersenne_twister_chi2.join(xoshiro256plusplus_chi2, on = index, how = "inner")
results_csv = results.to_csv(out_path + new_name)

In [7]:
%%latex
\begin{table}[]
\begin{tabular}{llll}
\rowcolor[HTML]{4472C4} 
{\color[HTML]{FFF} \textbf{k}} & {\color[HTML]{FFF} \textbf{chi\textasciicircum{}2   probability}} & {\color[HTML]{FFF} \textbf{observed   MT}} & {\color[HTML]{FFF} \textbf{observed XS256++}} \\
\rowcolor[HTML]{D9E1F2} 
1                              & 5.991                                                             & 0.292                                      & 0.068                                         \\
2                              & 7.815                                                             & 1.626                                      & 3.682                                         \\
\rowcolor[HTML]{D9E1F2} 
3                              & 9.488                                                             & 3.735                                      & 2.124                                         \\
4                              & 11.07                                                             & 4.255                                      & 4.525                                         \\
\rowcolor[HTML]{D9E1F2} 
5                              & 12.592                                                            & 2.86                                       & 6.345                                         \\
6                              & 14.067                                                            & 4.071                                      & 6.377                                         \\
\rowcolor[HTML]{D9E1F2} 
7                              & 15.507                                                            & 8.662                                      & 11.731                                        \\
8                              & 16.919                                                            & 14.426                                     & 8.693                                         \\
\rowcolor[HTML]{D9E1F2} 
9                              & 18.307                                                            & 11.732                                     & 6.932                                         \\
10                             & 19.675                                                            & 9.857                                      & 5.939                                         \\
\rowcolor[HTML]{D9E1F2} 
11                             & 21.026                                                            & 10.438                                     & 11.73                                         \\
12                             & 22.362                                                            & 10.985                                     & 15.543                                        \\
\rowcolor[HTML]{D9E1F2} 
13                             & 23.685                                                            & 22.603                                     & 12.022                                        \\
14                             & 24.996                                                            & 13.988                                     & 12.923                                        \\
\rowcolor[HTML]{D9E1F2} 
15                             & 26.296                                                            & 15.526                                     & 20.71                                         \\
16                             & 27.587                                                            & 16.288                                     & 11.501                                        \\
\rowcolor[HTML]{D9E1F2} 
17                             & 28.869                                                            & \cellcolor[HTML]{F8CBAD}32.26              & 12.478                                        \\
18                             & 30.144                                                            & \cellcolor[HTML]{F8CBAD}31.787             & 13.882                                       
\end{tabular}
\end{table}

<IPython.core.display.Latex object>

As we can see, both generators pass the $\chi^2$ test and we do not have to reject our null hypothesis $H_0$.

Visually, we can examine the generated buckets of both random generators in [TODO] the following plot. 

In [8]:
mers = "mersenne_"
xoshiro = "xoshiro_"
csv = ".csv"
angle = "alpha"
name_angles = "angles"

mers_random_angle = pd.read_csv(path + mers + name_angles + csv)
angle_mers = mers_random_angle[angle]
xs_random_angle = pd.read_csv(path + xoshiro + name_angles + csv)
angle_xs = xs_random_angle[angle]

with sns.plotting_context(sns.set()):
    x_size_per_plot = 7
    y_size_per_plot = 4
    rows = 4
    cols = 2
    
    fig, axes = plt.subplots(nrows=rows, ncols=cols, figsize=(x_size_per_plot * cols, y_size_per_plot * rows))
    cols = ["Mersenne Twister", "xoshiro256++"]
    for ax, col in zip(axes[0], cols):
        ax.annotate(col, xy=(0.5, 1), xytext=(0, 10),
                    textcoords='offset points', xycoords='axes fraction',
                    size='24', ha='center', va='baseline')

    axes[0,0].set_xlabel("5 buckets")
    axes[0,0].set_ylabel("frequency of angle")
    axes[0,0].hist(x=angle_mers,bins = 5)

    axes[0,1].set_xlabel("5 buckets")
    axes[0,1].hist(x=angle_xs, bins = 5)

    axes[1,0].set_xlabel("10 buckets")
    axes[1,0].set_ylabel("frequency of angle")
    axes[1,0].hist(x=angle_mers,bins = 10)

    axes[1,1].set_xlabel("10 buckets")
    axes[1,1].hist(x=angle_xs, bins = 10)

    axes[2,0].set_xlabel("50 buckets")
    axes[2,0].set_ylabel("frequency of angle")
    axes[2,0].hist(x=angle_mers,bins = 50)

    axes[2,1].set_xlabel("50 buckets")
    axes[2,1].hist(x=angle_xs, bins = 50)

    axes[3,0].set_xlabel("100 buckets")
    axes[3,0].set_ylabel("frequency of angle")
    axes[3,0].hist(x=angle_mers,bins = 100)

    axes[3,1].set_xlabel("100 buckets")
    axes[3,1].hist(x=angle_xs, bins = 100)

    plt.savefig("Assets/angle_buckets.png")
    plt.close()

![A histogram of different bucket sizes generated by MT and xoshiro256++](Assets/angle_buckets.png)

The Mersenne Twister has been known to fail certain statistical tests since its inception, by virtue of its mathematical characteristics. There exist other algorithms that are designed to be faster and that do not fail any known statistical tests.[@vigna2019] Ultimately, the xoshiro256++, developed by Sebastian Vigna and David Blackman, was used. It is a variant of the xorshift algorithm, which extends the bit-shift and xor methods by bitrotation, making it still very fast, and more "random" than the xorshift.[@wiki:xorshift][@unimi:xoshiro]

Note that testing a (pseudo) random number generator is usually much more involved than this, but since this has already been done extensively by other authors, we are satisfied with the $\chi^2$ test, simply to test the implementation of the xoshiro256++, since it plays an important part.[@wiki:prng][@vigna2019]

# Particle Streaming

The particle positions were drawn from a uniform real distribution in the interval $[0, 5)$ for the $x$-coordinate, and $[0, 1)$ for the $y$-coordinate. The velocities were initialized to be in the interval $[-1\% \cdot 5, 1\% \cdot 5)$ for the $v_x$ component, and $[-1\% \cdot 1, 1\% \cdot 1)$ for the $v_y$ component. The results can be seen [TODO] below. From the positions in the first row, the velocities in the second row, particle streaming is applied for 1 and 10 timesteps, respectively.

In [15]:
path = "../x64/Debug/Data/RNG/"
mers = "mersenne_"
xoshiro = "xoshiro_"
csv = ".csv"
particles = "particles"
moved_xy = "_after_move"
timesteps_moved_xy = "_after_xx_timesteps"
x = "x"
y = "y"
vx = "vx"
vy = "vy"

mers_particles = pd.read_csv(path + mers + particles + csv)
x_mers = mers_particles[x]
y_mers = mers_particles[y]
vx_mers = mers_particles[vx]
vy_mers = mers_particles[vy]

xs_particles = pd.read_csv(path + xoshiro + particles + csv)
x_xs = xs_particles[x]
y_xs = xs_particles[y]
vx_xs = xs_particles[vx]
vy_xs = xs_particles[vy]

mers_moved_particles = pd.read_csv(path + mers + particles + moved_xy + csv)
moved_x_mers = mers_moved_particles[x]
moved_y_mers = mers_moved_particles[y]
moved_vx_mers = mers_moved_particles[vx]
moved_vy_mers = mers_moved_particles[vy]

xs_moved_particles = pd.read_csv(path + xoshiro + particles + moved_xy + csv)
moved_x_xs = xs_moved_particles[x]
moved_y_xs = xs_moved_particles[y]
moved_vx_xs = xs_moved_particles[vx]
moved_vy_xs = xs_moved_particles[vy]

mers_timesteps_moved_particles = pd.read_csv(path + mers + particles + timesteps_moved_xy + csv)
timesteps_moved_x_mers = mers_timesteps_moved_particles[x]
timesteps_moved_y_mers = mers_timesteps_moved_particles[y]
timesteps_moved_vx_mers = mers_timesteps_moved_particles[vx]
timesteps_moved_vy_mers = mers_timesteps_moved_particles[vy]

xs_timesteps_moved_particles = pd.read_csv(path + xoshiro + particles + timesteps_moved_xy + csv)
timesteps_moved_x_xs = xs_timesteps_moved_particles[x]
timesteps_moved_y_xs = xs_timesteps_moved_particles[y]
timesteps_moved_vx_xs = xs_timesteps_moved_particles[vx]
timesteps_moved_vy_xs = xs_timesteps_moved_particles[vy]


with sns.plotting_context(sns.set()):
    point_size = 0.5
    x_size_per_plot = 7
    y_size_per_plot = 4
    rows = 4
    cols = 2
    #plt.figure(num = 1, figsize=(x_size_per_plot * cols, y_size_per_plot * rows))

    fig, axes = plt.subplots(nrows=rows, ncols=cols, figsize=(x_size_per_plot * cols, y_size_per_plot * rows))
    cols = ["Mersenne Twister", "xoshiro256++"]
    for ax, col in zip(axes[0], cols):
        ax.annotate(col, xy=(0.5, 1), xytext=(0, 10),
                    textcoords='offset points', xycoords='axes fraction',
                    size='24', ha='center', va='baseline')

    axes[0,0].set_ylabel(y)
    axes[0,0].plot(x_mers, y_mers, "o", markersize = point_size)
    axes[0,0].set_xlabel(x)

    axes[0,1].plot(x_xs, y_xs, "o", markersize = point_size)
    axes[0,1].set_xlabel(x)

    axes[1,0].set_ylabel("y-component of v")
    axes[1,0].plot(vx_mers, vy_mers, "o", markersize = point_size)
    axes[1,0].set_xlabel("x-component of v")

    axes[1,1].plot(vx_xs, vy_xs, "o", markersize = point_size)
    axes[1,1].set_xlabel("x-component of v")

    axes[2,0].set_ylabel(y)
    axes[2,0].plot(moved_x_mers, moved_y_mers, "o", markersize = point_size)
    axes[2,0].set_xlabel(x)

    axes[2,1].plot(moved_x_xs, moved_y_xs, "o", markersize = point_size)
    axes[2,1].set_xlabel(x)

    axes[3,0].set_ylabel(y)
    axes[3,0].plot(timesteps_moved_x_mers, timesteps_moved_y_mers, "o", markersize = point_size)
    axes[3,0].set_xlabel(x)

    axes[3,1].plot(timesteps_moved_x_xs, timesteps_moved_y_xs, "o", markersize = point_size)
    axes[3,1].set_xlabel(x)

    plt.savefig("Assets/particle_streaming.png")
    plt.close()
    
    # here something is wrong. these plots are the same.......

![Particle streaming without collision with MT and with xoshiro](Assets/particle_streaming.png)

We see the x and y coordinates are randomly initialized according to the shape of the container. Looking closely, one can see that our particles look very much like noise. The absolute value of the velocity components are initialized to at most 1% of their respective dimensions. After one timestep, some of the particles on the outer ranges have moved out of bounds, and after ten timesteps, the particles have thinned out considerably along the edges.

# The collision step

## Grid

Work in progress

In [19]:
path = "../x64/Debug/Data/"
csv = ".csv"
particles = "particles_"
timestep = 'timestep_'
x = 'x'
y = 'y'
vx = 'vx'
vy = 'vy'

import glob
filenames = glob.glob('{}{}{}*.csv'.format(path, particles, timestep))

import pandas as pd
dfs = []
for path in filenames:
    df = pd.read_csv(path)
    dfs.append(df)

x_columns = ['{}{}'.format(x, num) for num in range(0, len(filenames))]
y_columns = ['{}{}'.format(y, num) for num in range(0, len(filenames))]
vx_columns = ['{}{}'.format(vx, num) for num in range(0, len(filenames))]
vy_columns = ['{}{}'.format(vy, num) for num in range(0, len(filenames))]
    
x_evolution = pd.DataFrame(columns=x_columns)
y_evolution = pd.DataFrame(columns=y_columns)
vx_evolution = pd.DataFrame(columns=vx_columns)
vy_evolution = pd.DataFrame(columns=vy_columns)

i = 0
for df in dfs:
    x_evolution['{}{}'.format(x, i)] = df[x]
    y_evolution['{}{}'.format(y, i)] = df[y]
    vx_evolution['{}{}'.format(vx, i)] = df[vx]
    vy_evolution['{}{}'.format(vy, i)] = df[vy]
    i += 1

x_evolution.head()

        

,x0,x1,x2,x3,x4,x5,x6,x7,x8,x9,...,x990,x991,x992,x993,x994,x995,x996,x997,x998,x999
0,0.151566,0.158454,0.163691,0.161811,0.157700,0.164158,0.162847,0.157211,0.141097,0.141865,...,277.149,277.439,277.728,278.017,278.306,278.596,278.885,279.174,279.464,279.753
1,0.231102,0.228282,0.206493,0.201483,0.198738,0.196170,0.198016,0.190346,0.175495,0.190825,...,-127.901,-127.476,-127.051,-126.626,-126.201,-125.775,-125.350,-124.925,-124.500,-124.075
2,0.212257,0.222359,0.227338,0.232316,0.235197,0.237563,0.233138,0.229926,0.224070,0.280052,...,-119.888,-120.013,-120.139,-120.265,-120.390,-120.516,-120.642,-120.767,-120.893,-121.019
3,0.695957,0.696264,0.707172,0.705697,0.716855,0.721179,0.752653,0.771790,0.790926,0.819962,...,-845.226,-845.765,-846.304,-846.842,-847.381,-847.920,-848.458,-848.997,-849.536,-850.074
4,0.269001,0.271747,0.275402,0.277689,0.279587,0.278236,0.277517,0.280980,0.284660,0.303956,...,-220.558,-221.244,-221.930,-222.615,-223.301,-223.987,-224.673,-225.359,-226.045,-226.731


In [22]:
vx_evolution.sum(axis = 0)

vx0      -0.642801
vx1      -0.627699
vx2      -0.986298
vx3      -1.759460
vx4       0.519716
           ...    
vx995    93.962144
vx996    89.461289
vx997    88.255094
vx998    88.740288
vx999    88.801907
Length: 1000, dtype: float64

In [23]:
vy_evolution.sum(axis = 0)

vy0       -0.221832
vy1       -0.466422
vy2       -1.051625
vy3       -1.415851
vy4       -1.297967
            ...    
vy995   -114.254012
vy996   -109.042020
vy997   -107.853837
vy998   -108.138853
vy999   -108.057055
Length: 1000, dtype: float64

['x0',
 'x1',
 'x2',
 'x3',
 'x4',
 'x5',
 'x6',
 'x7',
 'x8',
 'x9',
 'x10',
 'x11',
 'x12',
 'x13',
 'x14',
 'x15',
 'x16',
 'x17',
 'x18',
 'x19',
 'x20',
 'x21',
 'x22',
 'x23',
 'x24',
 'x25',
 'x26',
 'x27',
 'x28',
 'x29',
 'x30',
 'x31',
 'x32',
 'x33',
 'x34',
 'x35',
 'x36',
 'x37',
 'x38',
 'x39',
 'x40',
 'x41',
 'x42',
 'x43',
 'x44',
 'x45',
 'x46',
 'x47',
 'x48',
 'x49',
 'x50',
 'x51',
 'x52',
 'x53',
 'x54',
 'x55',
 'x56',
 'x57',
 'x58',
 'x59',
 'x60',
 'x61',
 'x62',
 'x63',
 'x64',
 'x65',
 'x66',
 'x67',
 'x68',
 'x69',
 'x70',
 'x71',
 'x72',
 'x73',
 'x74',
 'x75',
 'x76',
 'x77',
 'x78',
 'x79',
 'x80',
 'x81',
 'x82',
 'x83',
 'x84',
 'x85',
 'x86',
 'x87',
 'x88',
 'x89',
 'x90',
 'x91',
 'x92',
 'x93',
 'x94',
 'x95',
 'x96',
 'x97',
 'x98',
 'x99',
 'x100',
 'x101',
 'x102',
 'x103',
 'x104',
 'x105',
 'x106',
 'x107',
 'x108',
 'x109',
 'x110',
 'x111',
 'x112',
 'x113',
 'x114',
 'x115',
 'x116',
 'x117',
 'x118',
 'x119',
 'x120',
 'x121',
 'x122',
 'x

## Velocity updating

The velocity of a particle updates according to
\begin{equation}
\vect{v_{i}} \rightarrow \vect{V_c} + \matr{R}(\alpha)[\vect{v_i} - \vect{V_c}],
\end{equation}

where $\vect{v_{i}}$ is the velocity of the $i-th$ particle, $\vect{V_c}$ is the mean velocity of all particles belonging to cell $c$, specified by $i$'s position. The matrix

$$
R(\alpha) = 
\left[ \begin{array}{rr}
cos(\alpha) & -sin(\alpha) \\
sin(\alpha) & cos(\alpha) \\
\end{array}\right]
$$

is a simple 2d-rotation matrix. The angle $\alpha$ is uniformly sampled from the interval $[0, 2\pi)$ on a per-cell basis.[@malev1999]

<!--

CONSIDER THIS. SHOULD BECOME MAXWELL WHEN IMPLEMENTING COLLISION STEP TOO
After having implemented the streaming step and that other one: after time driftting, looks like this: (+velocity distribution = maxwell?)

### some equations to copy

$$
r(t + \Delta t) = r(t) + \Delta t \cdot v(t)
$$

## Converting to Word doc (others possible too, f.ex. .tex)

In [ ]:
# just do it manually, it works on anaconda env datascience

import subprocess
#automatic document conversion to markdown and then to word
#first convert the ipython notebook paper.ipynb to markdown
subprocess.run("jupyter nbconvert --to markdown thesis.ipynb --output-dir='./Generated'") #--output-dir='./Generated'
#next remove code
path = "./Generated/thesis.md"
with open(path, "r") as f:
    lines = f.readlines()
    idx = []
    idx_files = []
    for i, line in enumerate(lines):
        if (line.startswith("```")):
            idx.append(i)
        if ("thesis_files" in line):
            c = line.find("thesis_files")
            lines[i] = line[0:c] + "" + line[c:]

idx = sorted(idx, reverse=True) # reverse order so not deleting lines and then missing others
for current, previous in zip(idx[::2], idx[1::2]):
    print("Deleting {p}:{c}".format(p=previous, c=current+1))
    print('\n'.join(lines[previous:current+1]))
    del lines[previous:current+1]
    
with open(path, "w") as f:
    #f.write("\\newcommand{\matr}[1]\\textbf{#1}")
    #f.write("\\newcommand{\\vect}[1]{\\vec{#1}}")
    for line in lines:
        f.write("%s" % line)
#next convert markdown to ms word
conversion_tex = "pandoc -s ./Generated/thesis.md -o ./Generated/thesis.tex --filter pandoc-citeproc --bibliography=\"list.bib\" --csl=\"apa.csl\""
subprocess.run(conversion_tex)
conversion_pdf = "pandoc -s ./Generated/thesis.md -o ./Generated/thesis.pdf --filter pandoc-citeproc --bibliography=\"list.bib\" --csl=\"apa.csl\""
subprocess.run(conversion_pdf)
# LATEX TO DOCX pandoc -s math.tex -o example30.docx

## Equation Numbering jupyter extension
conda install -c conda-forge jupyter_contrib_nbextensions

jupyter contrib nbextension install --user

jupyter nbextension enable equation-numbering/main

### Turn equation numbering on/off

In [ ]:
%%javascript
MathJax.Hub.Config({
    TeX: { equationNumbers: { autoNumber: "AMS" } }
});

### Renumber equations

In [ ]:
%%javascript
MathJax.Hub.Queue(
  ["resetEquationNumbers", MathJax.InputJax.TeX],
  ["PreProcess", MathJax.Hub],
  ["Reprocess", MathJax.Hub]
);

-->